In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:   
#%sql duckdb://///home/patrizio115/Documents/dmeyf.db

In [ ]:
%%sql
select 'hola' as col

In [ ]:
%%sql

create or replace table competencia_03 as 

select 
    *
from read_csv_auto('~/buckets/b1/datasets/competencia_03_crudo.csv.gz')

In [ ]:
%%sql 

select distinct foto_mes from competencia_03

In [ ]:
%sql select count(*) from competencia_03

In [ ]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_03
), clientes as (
    select distinct numero_de_cliente from competencia_03
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , 
        case
        WHEN ((mes_0 == 0) or (mes_1 is NULL) or (mes_2 is NULL)) THEN NULL
        WHEN (mes_1 == 0) THEN 'BAJA+1'
        WHEN (mes_2 == 0) THEN 'BAJA+2'
        ELSE 'CONTINUA'
        END as clase_ternaria
    from todo t
    left join competencia_03 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria 
where mes_0 = 1

In [ ]:
%%sql
alter table competencia_03 add column clase_ternaria VARCHAR(10)

In [ ]:
%%sql
update competencia_03
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_03.numero_de_cliente = targets.numero_de_cliente and competencia_03.foto_mes = targets.foto_mes;

In [ ]:
%%sql

select 
    clase_ternaria, 
    count(*) as cant,
from competencia_03
group by clase_ternaria

In [ ]:
%%sql
COPY competencia_03 TO '~/buckets/b1/datasets/competencia_03.csv.gz' (FORMAT CSV, COMPRESSION GZIP, HEADER)

In [ ]:
%%sql

SELECT 
    foto_mes, 
    clase_ternaria,
    COUNT(*) as cantidad,
FROM competencia_03
WHERE foto_mes in (202101, 202102, 202103, 202104, 202105)
GROUP BY 1,2
ORDER BY 1,2 ASC